In [1]:
import pandas as pd

%load_ext autoreload
%autoreload 2

folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms'
csv_folder = '{}/csvs_for_paper'.format(folder)
bed_folder = '{}/bed'.format(folder)
# folder2 = '/projects/ps-yeolab2/obotvinnik/singlecell_pnm'

splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)
alternative_feature_folder = '{}/alternative'.format(splicing_feature_folder)
constitutive_feature_folder = '{}/constitutive'.format(splicing_feature_folder)

alt_exons_bedfile = '{}/exons.bed'.format(alternative_feature_folder)
constitutive_bedfile = '{}/exons.bed'.format(constitutive_feature_folder)


# splicing_feature_data = pd.read_csv('{}/splicing_feature_data.csv'.format(csv_folder), index_col=0)
# print(splicing_feature_data.shape)
# splicing_feature_data.head()

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())


In [2]:
import pybedtools

In [3]:
alt_exons = pybedtools.BedTool(alt_exons_bedfile)
constitutive_exons = pybedtools.BedTool(constitutive_bedfile)

In [4]:
print(len(alt_exons), len(constitutive_exons))

(43903, 5893)


## Caclulate upstream nucleotide fraction

In [5]:
hg19_fasta = '/projects/ps-yeolab/genomes/hg19/chromosomes/all.fa'

In [14]:
from Bio import SeqIO

def calculate_nucleotide_content(bed, window, nt, genome, fasta, nucleotide, context, intersect=None):
    """Count the percentage of nucleotides in windows flanking a bed file (e.g. exons)
    
    Parameters
    ----------
    bed : pybedtools.BedTool
        BedTool object of exons whose flanking nucleotide content you want to measure
    window : int
        Size of the windows to use
    nt : int
        Size of the flanking region
    genome : str
        Unique name of the genome to look up chromosome sizes, or the chromosome sizes themselves
    fasta : str
        Location of the genome sequences as a fasta file
    nucleotide : str
        Which nucleotide you're counting
    context : 'upstream' | 'downstream'
        Relative to the strand, whether you want the upstream or downstream sequences
    intersect = pybedtools.BedTool, optional
        Another bed file to intersect with if you're only interested in a subset of regions (e.g. only conserved elemets)
    
    Returns
    -------
    nucleotide_fraction : pandas.DataFrame
        
    
    """
    windows = nt/window

    index = [x.name for x in bed]

    nucleotide_fractions = []
    if context.startswith('up'):
        # Upstream - go left (upstream) the window size, and right zero
        flank_kws = dict(l=window, r=0, s=True, genome=genome)
    elif context.startswith('d'):
        # downstream - go right (downstream) the window size, and right zero
        flank_kws = dict(l=0, r=window, s=True, genome=genome)

    for i in range(windows):
        if i == 0:
            subset = bed.flank(**flank_kws)
        else:
            subset = subset.flank(**flank_kws)
            
        if intersect is not None:
            subset = subset.intersect(intersect)
            import pdb; pdb.set_trace()
            df = subset.to_dataframe()
            index = df['name']

        seqs = subset.sequence(fi=fasta, s=True)
        with open(seqs.seqfn) as f:
            records = list(SeqIO.parse(f, 'fasta'))
        nucleotide_fraction = [sum(1 for x in s.seq if x == nucleotide)/float(window) for s in records]
        nucleotide_fraction = pd.Series(nucleotide_fraction, index=index).reset_index()
        nucleotide_fractions.append(nucleotide_fraction)
    nucleotide_fraction_df = pd.concat(nucleotide_fractions, axis=1)
    return nucleotide_fraction_df

In [15]:
csv_folder = '/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper'

In [16]:
import six

In [17]:
splicing_feature_folder = '{}/splicing_feature_data'.format(csv_folder)

In [19]:
%pdb

Automatic pdb calling has been turned ON


In [20]:
placental = '/projects/ps-yeolab/genomes/hg19/database/phastConsElements46wayPlacental.bed'

nucleotides = 'ACGT'
contexts = 'upstream', 'downstream'
nt = 200

alt_exons = pybedtools.BedTool(alt_exons_bedfile)
constitutive_exons = pybedtools.BedTool(constitutive_bedfile)
print(len(alt_exons), len(constitutive_exons))

beds = {'alternative': alt_exons, 'constitutive': constitutive_exons}

for nucleotide in nucleotides:
    for context in contexts:
        for name, bed in beds.items():
            six.print_(nucleotide, context, name)
            nucleotide_folder = '{}/{}/nucleotide_content'.format(splicing_feature_folder, name)
            ! mkdir $nucleotide_folder
            nucleotide_content = calculate_nucleotide_content(bed, window=10, nt=nt, genome='hg19', fasta=hg19_fasta, 
                                                             nucleotide=nucleotide, context=context, intersect=placental)
            nucleotide_content.to_csv('{}/{}{}_nucleotide_content_{}_placental_mammal.csv'.format(
                    nucleotide_folder, context, nt, nucleotide), index=False)

(43903, 5893)
A upstream alternative
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/nucleotide_content': File exists
> <ipython-input-14-5d1ee3189aac>(52)calculate_nucleotide_content()
-> df = subset.to_dataframe()
(Pdb) dir()
['bed', 'context', 'fasta', 'flank_kws', 'genome', 'i', 'index', 'intersect', 'nt', 'nucleotide', 'nucleotide_fractions', 'pdb', 'subset', 'window', 'windows', 'x']
(Pdb) subset
<BedTool(/state/partition1/obotvinnik/4944699.tscc-mgr.local/pybedtools.6opDQG.tmp)>
(Pdb) df
*** NameError: name 'df' is not defined
(Pdb) n
> <ipython-input-14-5d1ee3189aac>(53)calculate_nucleotide_content()
-> index = df['name']
(Pdb) df.head()
   chrom      start        end  \
0  chr10  101997988  101997990   
1  chr10  102010089  102010091   
2  chr10  102020076  102020078   
3  chr10  102020085  102020086   
4  chr10  102020076  102020078   

                                                name  score s

BdbQuit: 

> /home/obotvinnik/anaconda/lib/python2.7/bdb.py(68)dispatch_line()
     66         if self.stop_here(frame) or self.break_here(frame):
     67             self.user_line(frame)
---> 68             if self.quitting: raise BdbQuit
     69         return self.trace_dispatch
     70 

ipdb> q


In [12]:
nucleotides = 'ACGT'
contexts = 'upstream', 'downstream'

alt_exons = pybedtools.BedTool(alt_exons_bedfile)
constitutive_exons = pybedtools.BedTool(constitutive_bedfile)
print(len(alt_exons), len(constitutive_exons))

beds = {'alternative': alt_exons, 'constitutive': constitutive_exons}

for nucleotide in nucleotides:
    for context in contexts:
        for name, bed in beds.items():
            six.print_(nucleotide, context, name)
            nucleotide_folder = '{}/{}/nucleotide_content'.format(splicing_feature_folder, name)
            ! mkdir $nucleotide_folder
            nucleotide_content = calculate_nucleotide_content(bed, window=10, nt=nt, genome='hg19', fasta=hg19_fasta, 
                                                             nucleotide=nucleotide, context=context)
            nucleotide_content.to_csv('{}/{}{}_nucleotide_content_{}.csv'.format(
                    nucleotide_folder, context, nt, nucleotide))

(43903, 5893)
A upstream alternative
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/nucleotide_content': File exists
A upstream constitutive
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/nucleotide_content': File exists
A downstream alternative
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/nucleotide_content': File exists
A downstream constitutive
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/constitutive/nucleotide_content': File exists
C upstream alternative
mkdir: cannot create directory `/projects/ps-yeolab/obotvinnik/singlecell_pnms/csvs_for_paper/splicing_feature_data/alternative/nucleotide_content': File exists
C upstream constitutive
mkdir: cannot create directory `/p

In [13]:
nucleotide_content.head()

,index,0,index,0,index,0,index,0,index,0,...,index,0,index,0,index,0,index,0,index,0
0,exon:chr10:102039881-102039985:-,0.3,exon:chr10:102039881-102039985:-,0.6,exon:chr10:102039881-102039985:-,0.6,exon:chr10:102039881-102039985:-,0.2,exon:chr10:102039881-102039985:-,0.4,...,exon:chr10:102039881-102039985:-,0.0,exon:chr10:102039881-102039985:-,0.0,exon:chr10:102039881-102039985:-,0.0,exon:chr10:102039881-102039985:-,0.0,exon:chr10:102039881-102039985:-,0.0
1,exon:chr10:102040691-102040810:-,0.4,exon:chr10:102040691-102040810:-,0.5,exon:chr10:102040691-102040810:-,0.3,exon:chr10:102040691-102040810:-,0.7,exon:chr10:102040691-102040810:-,0.7,...,exon:chr10:102040691-102040810:-,0.4,exon:chr10:102040691-102040810:-,0.5,exon:chr10:102040691-102040810:-,0.6,exon:chr10:102040691-102040810:-,0.6,exon:chr10:102040691-102040810:-,0.4
2,exon:chr10:102045854-102045970:-,0.1,exon:chr10:102045854-102045970:-,0.4,exon:chr10:102045854-102045970:-,0.3,exon:chr10:102045854-102045970:-,0.3,exon:chr10:102045854-102045970:-,0.1,...,exon:chr10:102045854-102045970:-,0.1,exon:chr10:102045854-102045970:-,0.2,exon:chr10:102045854-102045970:-,0.3,exon:chr10:102045854-102045970:-,0.3,exon:chr10:102045854-102045970:-,0.1
3,exon:chr10:102286732-102286831:-,0.1,exon:chr10:102286732-102286831:-,0.3,exon:chr10:102286732-102286831:-,0.5,exon:chr10:102286732-102286831:-,0.6,exon:chr10:102286732-102286831:-,0.6,...,exon:chr10:102286732-102286831:-,0.1,exon:chr10:102286732-102286831:-,0.2,exon:chr10:102286732-102286831:-,0.3,exon:chr10:102286732-102286831:-,0.3,exon:chr10:102286732-102286831:-,0.2
4,exon:chr10:102289123-102289263:-,0.3,exon:chr10:102289123-102289263:-,0.1,exon:chr10:102289123-102289263:-,0.2,exon:chr10:102289123-102289263:-,0.1,exon:chr10:102289123-102289263:-,0.3,...,exon:chr10:102289123-102289263:-,0.1,exon:chr10:102289123-102289263:-,0.1,exon:chr10:102289123-102289263:-,0.6,exon:chr10:102289123-102289263:-,0.4,exon:chr10:102289123-102289263:-,0.4
